In [6]:
import requests
from bs4 import BeautifulSoup
import time
import json
from tqdm import tqdm

In [7]:
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36"}
session = requests.Session()

article_urls = []
category = []
headers_list = []
article_summary = []
author_list = []
date_list = []

# Step 1: Collect article urls and summaries
for page in tqdm(range(1, 2)):
    page_url = f'https://www.snexplores.org/all-stories/page/{page}'
    resp = session.get(page_url, headers=headers)
    soup = BeautifulSoup(resp.content, 'html.parser')
    if not soup.find_all('div', class_='post-item-river__content___Y9C70'):
        break
    blocks = soup.find_all('div', class_='post-item-river__content___Y9C70')
    category.extend([block.find('a').get_text(strip=True) for block in blocks])
    headers_list.extend([block.find('h3').get_text(strip=True) for block in blocks])
    author_list.extend([block.find('span').get_text(strip=True) for block in blocks])
    date_list.extend([block.find('time').get_text(strip=True) for block in blocks])
    article_urls.extend([block.find_all('a')[1].get('href') for block in blocks])
    article_summary.extend([block.find('p').get_text(strip=True) if block.find('p') else '' for block in blocks])
    time.sleep(2)

# Step 2: Loop over article urls and scrape information
articles = []
for url in tqdm(article_urls):
    index = article_urls.index(url)
    resp = session.get(url, headers=headers)
    soup = BeautifulSoup(resp.content, 'lxml')
    
    article = {
        "url": url,
        "category": category[index],
        "header": headers_list[index],
        "summary": article_summary[index],
        "author": author_list[index],
        "date": date_list[index],
        "article_text": " ".join(p.get_text(strip=True) for p in soup.select('div.rich-text.single__rich-text___DT62t p') if not p.find_parent('figure'))
    }
    
    
    try:
        article["power_words"] = soup.find('div', class_='power-words-container').get_text(strip=True)
    except AttributeError:
        article["power_words"] = None
    
    articles.append(article)
    time.sleep(2)


100%|██████████| 12/12 [00:34<00:00,  2.92s/it]


In [8]:
with open("articles.json", "w", encoding="UTF-8") as f:
    json.dump(articles, f, ensure_ascii=False)

In [9]:
import pandas as pd
df = pd.read_json('articles.json')
pd.set_option('display.max_colwidth', 200)

In [15]:
df['article_text'][1][0:200]

'This is another in our new series identifying technologies and actions that can slow climate change, reduce its impacts or help communities cope with a rapidly changing world. CHICAGO— Native people i'

In [17]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-QuestionAnswer")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-QuestionAnswer")

context = df['article_text'][1][0:300]
print(context)
inputs = tokenizer(context, return_tensors="pt")
outputs = model.generate(**inputs, max_length=100)
question_answer = tokenizer.decode(outputs[0], skip_special_tokens=False)
question_answer = question_answer.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
question, answer = question_answer.split(tokenizer.sep_token)

print("question:", question)
print("answer:", answer)


: 

: 